In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from IPython.display import Image


In [4]:
import torch
import numpy as np
import os
import torch.nn as nn
from tqdm import tqdm
import json
from functools import partial
from torch import einsum, nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from einops import pack, rearrange, reduce, repeat, unpack


In [5]:
from core.datasets.text_encoders import T5Conditioner,MPNETConditioner

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helpers

In [85]:
from configs.config import cfg, get_cfg_defaults
def load_vqvae(gen_cfg):

    body_cfg = get_cfg_defaults()
    body_cfg.merge_from_file(gen_cfg.vqvae.body_config)
    body_model = (
        instantiate_from_config(body_cfg.vqvae).to(device).eval()
    )
    body_model.load(os.path.join(body_cfg.output_dir, "vqvae_motion.pt"))

    if gen_cfg.vqvae.left_hand_config is  None and gen_cfg.vqvae.right_hand_config is None:
        return body_model, body_cfg
    
    if gen_cfg.vqvae.left_hand_config is not None:
        left_cfg = get_cfg_defaults()
        left_cfg.merge_from_file(gen_cfg.vqvae.left_hand_config)
        left_hand_model = instantiate_from_config(left_cfg.vqvae).to(device).eval()
        left_hand_model.load(
            os.path.join(left_cfg.output_dir, "vqvae_motion.pt")
        )
    else:
        left_hand_model = None
        
    if gen_cfg.vqvae.right_hand_config is not None:
        right_cfg = get_cfg_defaults()
        right_cfg.merge_from_file(gen_cfg.vqvae.right_hand_config)
        right_hand_model = instantiate_from_config(right_cfg.vqvae).to(device).eval()
        right_hand_model.load(
            os.path.join(right_cfg.output_dir, "vqvae_motion.pt")
        )
    else:
        right_hand_model = None

    return body_model, left_hand_model , right_hand_model , body_cfg , left_cfg , right_cfg

def bkn_to_motion( codes, dset , remove_translation = True):
    # codes b k n

    k = codes.shape[1]
    mrep = dset.motion_rep

    if k == 1:
        if mrep == MotionRep("body"):

            body_inds = codes[:, 0]
            body_motion = body_model.decode(body_inds[0:1]).detach().cpu()

            if remove_translation:
                z = torch.zeros(
                    body_motion.shape[:-1] + (2,),
                    dtype=body_motion.dtype,
                    device=body_motion.device,
                )
                body_motion = torch.cat(
                    [body_motion[..., 0:1], z, body_motion[..., 1:]], -1
                )

            body_M = dset.toMotion(
                body_motion[0],
                motion_rep=MotionRep("body"),
                hml_rep=body_cfg.dataset.hml_rep,
            )

            return body_M

        elif mrep == MotionRep("left_hand"):

            left_inds = codes[:, 0]
            left_motion = left_hand_model.decode(left_inds[0:1]).detach().cpu()
            left_M = dset.toMotion(
                left_motion[0],
                motion_rep=MotionRep(left_cfg.dataset.motion_rep),
                hml_rep=left_cfg.dataset.hml_rep,
            )
            return left_M

        elif mrep == MotionRep("right_hand"):
            right_inds = codes[:, 0]
            right_motion = (
                right_hand_model.decode(right_inds[0:1]).detach().cpu()
            )
            right_M = dset.toMotion(
                right_motion[0],
                motion_rep=MotionRep(right_cfg.dataset.motion_rep),
                hml_rep=right_cfg.dataset.hml_rep,
            )
            return right_M

    if k == 2:
        left_inds = codes[:, 0]
        right_inds = codes[:, 1]

        left_motion = left_hand_model.decode(left_inds[0:1]).detach().cpu()
        right_motion = right_hand_model.decode(right_inds[0:1]).detach().cpu()

        left_M = dset.toMotion(
            left_motion[0],
            motion_rep=MotionRep(left_cfg.dataset.motion_rep),
            hml_rep=left_cfg.dataset.hml_rep,
        )
        right_M = dset.toMotion(
            right_motion[0],
            motion_rep=MotionRep(right_cfg.dataset.motion_rep),
            hml_rep=right_cfg.dataset.hml_rep,
        )
        hand_M = left_M + right_M
        hand_M.motion_rep = MotionRep.HAND
        hand_M.hml_rep = "".join(
            [i for i in left_M.hml_rep if i in right_M.hml_rep]
        )
        return hand_M

    if k == 3:
        left_inds = codes[:, 1]
        right_inds = codes[:, 2]
        body_inds = codes[:, 0]
        body_motion = body_model.decode(body_inds[0:1]).detach().cpu()

        
        if remove_translation:
            z = torch.zeros(
                body_motion.shape[:-1] + (2,),
                dtype=body_motion.dtype,
                device=body_motion.device,
            )
            body_motion = torch.cat([body_motion[..., 0:1], z, body_motion[..., 1:]], -1)

        left_motion = left_hand_model.decode(left_inds[0:1]).detach().cpu()
        right_motion = right_hand_model.decode(right_inds[0:1]).detach().cpu()

        body_M = dset.toMotion(
            body_motion[0],
            motion_rep=MotionRep("body"),
            hml_rep = body_cfg.dataset.hml_rep)

        left_M = dset.toMotion(
            left_motion[0],
            motion_rep=MotionRep("left_hand"),
            hml_rep=left_cfg.dataset.hml_rep)
        right_M = dset.toMotion(
            right_motion[0],
            motion_rep=MotionRep("right_hand"),
            hml_rep=right_cfg.dataset.hml_rep)
        full_M = dset.to_full_joint_representation(body_M, left_M, right_M)
        return full_M



## TMR

In [41]:
from core.models.utils import instantiate_from_config, get_obj_from_str
from core.datasets.tmr_dataset import TMRDataset, load_dataset, simple_collate
from core import MotionRep, TextRep, AudioRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser
from configs.config_tmr import get_cfg_defaults as get_cfg_defaults_tmr
from core.models.TMR.tmr import TMR
from core.models.TMR.tmr import get_score_matrix


In [8]:
tmr_cfg = get_cfg_defaults_tmr()
tmr_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/tmr/tmr.yaml")
tmr_cfg.freeze()
dataset_args = tmr_cfg.dataset
tmr_parms = tmr_cfg.tmr

In [9]:
_ = tmr_parms.pop("target")
motion_encoder = instantiate_from_config(tmr_cfg.motion_encoder).to(device)
text_encoder = instantiate_from_config(tmr_cfg.text_encoder).to(device)
motion_decoder = instantiate_from_config(tmr_cfg.motion_decoder).to(device)
tmr = TMR(motion_encoder , text_encoder , motion_decoder , lr = tmr_cfg.train.learning_rate, **tmr_parms).to(device).eval()

In [10]:
pkg = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/tmr/tmr.pt")
tmr.load_state_dict(pkg["model"])

In [ ]:
def get_latents(inputs , conditions):
    text_conds = conditions["text"]
    text_x_dict = {"x": text_conds[0], "mask": text_conds[1].to(torch.bool)}
    motion_x_dict = {"x": inputs[0], "mask": inputs[1].to(torch.bool)}
    motion_mask = motion_x_dict["mask"]
    text_mask = text_x_dict["mask"]
    t_motions, t_latents, t_dists = tmr(
            text_x_dict, mask=motion_mask, return_all=True
        )

    # motion -> motion
    m_motions, m_latents, m_dists = tmr(
        motion_x_dict, mask=motion_mask, return_all=True
    )
    return t_latents , m_latents

In [24]:
train_ds, sampler_train, weights_train  = load_dataset(dataset_names = ["idea400" , "choreomaster"] , dataset_args=dataset_args, split = "test")


Total number of motions idea400: 577 and texts 577
Total number of motions choreomaster: 2 and texts 2


In [25]:
condition_provider = ConditionProvider(
            text_conditioner_name = dataset_args.text_conditioner_name,
            motion_rep=MotionRep(dataset_args.motion_rep),
            audio_rep=AudioRep(dataset_args.audio_rep),
            text_rep=TextRep(dataset_args.text_rep),
            motion_padding=dataset_args.motion_padding,
            motion_max_length_s=dataset_args.motion_max_length_s,
            fps=30,
            # only_motion = True
        )

In [28]:
train_loader = torch.utils.data.DataLoader(
        train_ds,
        4,
        # sampler=sampler_train,
        # shuffle = False,
        collate_fn=partial(simple_collate , conditioner = condition_provider),
        # drop_last=True,
    )

In [29]:
for inputs, conditions in (train_loader):
    break
    

In [15]:
motion = inputs["motion"][0]
motion_mask = inputs["motion"][1].to(torch.bool)
text = conditions["text"][0]
text_mask = conditions["text"][1].to(torch.bool)

In [45]:
losses = tmr.compute_loss(inputs["motion"] , conditions)
losses

In [39]:
t_latents , m_latents = get_latents(inputs["motion"],conditions)
score = get_score_matrix(t_latents, m_latents).cpu()

### Eval

In [81]:
from core.models.TMR.tmr import get_score_matrix
import random

In [11]:
condition_provider = ConditionProvider(
            text_conditioner_name = dataset_args.text_conditioner_name,
            motion_rep=MotionRep(dataset_args.motion_rep),
            audio_rep=AudioRep(dataset_args.audio_rep),
            text_rep=TextRep(dataset_args.text_rep),
            motion_padding=dataset_args.motion_padding,
            motion_max_length_s=dataset_args.motion_max_length_s,
            fps=30,
            # only_motion = True
        )

In [12]:
from core.datasets.base_dataset import BaseMotionDataset
base_dset = BaseMotionDataset(motion_rep=MotionRep.FULL , hml_rep= "gprvc")

In [115]:
def getScore(text:str , motion_path:str, return_embeds = False):
    motion = np.load(motion_path)

    text_embed, text_mask = condition_provider._get_text_features(
        raw_text=text,
    )
    processed_motion = base_dset.get_processed_motion(
            motion, motion_rep=MotionRep(dataset_args.motion_rep), hml_rep=dataset_args.hml_rep
        )
    mot_body = torch.Tensor(processed_motion()).to(device)[None]
    print(mot_body.shape)
    motion_x_dict = {"x": mot_body, "mask": torch.ones_like(mot_body)[...,0].to(torch.bool)}
    text_x_dict = {"x": text_embed, "mask": text_mask.to(torch.bool)}
    with torch.no_grad():
        lat_m = tmr.encode(motion_x_dict, sample_mean=True)[0]
        lat_t = tmr.encode(text_x_dict, sample_mean=True)[0]
        score = get_score_matrix(lat_t, lat_m).cpu()

    if return_embeds:
        return score ,lat_m ,lat_t
    return score

In [121]:
text="a man sets to do a backflips then fails back flip and falls to the ground"
mot="/srv/hays-lab/scratch/sanisetty3/motionx/motion_data/new_joint_vecs/humanml/001034.npy"
text2 = "a person standing loses balance falling to the right and recovers standing"
mot2="/srv/hays-lab/scratch/sanisetty3/motionx/motion_data/new_joint_vecs/humanml/000012.npy"

In [122]:
motion = np.load(mot)
motion2 = np.load(mot2)

In [123]:
getScore(text , motion)

torch.Size([1, 120, 137])


tensor(0.7139)

In [124]:
getScore(text2 , motion2)

torch.Size([1, 120, 137])


tensor(0.9692)

## Muse Gen

In [64]:
from core.models.generation.muse2 import generate_animation
from core import MotionTokenizerParams, pattern_providers

from core.param_dataclasses import pattern_providers
from core.datasets.multimodal_dataset import MotionIndicesAudioTextDataset, load_dataset_gen, simple_collate
from core.models.utils import instantiate_from_config, get_obj_from_str
from core import MotionRep, AudioRep, TextRep
from core.datasets.conditioner import ConditionProvider,ConditionFuser
from core.models.generation.muse2 import MotionMuse as MotionMuse2
import einops
from configs.config_t2m import get_cfg_defaults as muse_get_cfg_defaults
from core import MotionTokenizerParams
import math

In [47]:
gen_cfg = muse_get_cfg_defaults()
gen_cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_muse_body_hands/motion_muse_body_hands.yaml")
gen_cfg.freeze()
tranformer_config = gen_cfg.motion_generator
fuse_config = gen_cfg.fuser
pattern_config = gen_cfg.codebooks_pattern
dataset_args = gen_cfg.dataset

target = tranformer_config.pop("target")
motion_gen = MotionMuse2(tranformer_config , fuse_config , pattern_config).to(device).eval()
pkg = torch.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_muse_body_hands/motion_muse.pt", map_location="cuda")
motion_gen.load_state_dict(pkg["model"])

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


<All keys matched successfully>

In [48]:
body_model, left_hand_model , right_hand_model , body_cfg , left_cfg , right_cfg = load_vqvae(gen_cfg)

Sync is turned on False
loaded model with  0.03015906736254692 tensor([110000.], device='cuda:0') steps


In [49]:
from core.datasets.base_dataset import BaseMotionDataset
base_dset = BaseMotionDataset(motion_rep=MotionRep.BODY , hml_rep= "gpvc")

In [ ]:
condition_provider.audio_dim = 128

In [112]:
inputs["lens"].min()

tensor(183.)

In [100]:
text_ = inputs["texts"][-1]
duration_s = inputs["lens"][-1]//30

In [113]:
duration_s = int(inputs["lens"].min()//30)

In [115]:
all_ids_body = generate_animation(motion_gen , condition_provider ,overlap = 15, duration_s = duration_s ,  text = text_ , use_token_critic = True, timesteps = 24 )
gen_motion = bkn_to_motion(all_ids_body[:,:1], base_dset)

100%|███████████████████████████████████████████████████████████████████| 72/72 [00:01<00:00, 37.02it/s]


In [109]:
# gen_motion = bkn_to_motion(all_ids_body[:,:1], base_dset)
# base_dset.render_hml(
#                     gen_motion,
#                     f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/gen_novel_full.gif",
#                     zero_trans = True,
#                     zero_orient = True,
    
#                 )
# Image(open(f"/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/render/gen_novel_full.gif",'rb').read())

In [118]:
gen_motion().shape

torch.Size([180, 137])

### Eval script

In [117]:
inputs["motion"][0].shape

torch.Size([4, 299, 137])

In [129]:
motion_list = []
motion_pred_list = []
R_precision_real = 0
R_precision = 0
matching_score_real = 0
matching_score_pred = 0
with torch.no_grad():
    
    t_latents , m_latents = get_latents(inputs["motion"],conditions)
    score = get_score_matrix(t_latents, m_latents).cpu()

    pred_pose_eval = torch.zeros_like(inputs["motion"][0])
    for k in range(4):
        lenn = int(inputs["lens"][k])
        text_ = inputs["texts"][k]
        duration_s = int(lenn//30)
        all_ids_body = generate_animation(motion_gen , condition_provider ,overlap = 15, duration_s = duration_s ,  text = text_ , use_token_critic = True, timesteps = 24 )
        gen_motion = bkn_to_motion(all_ids_body[:,:1], base_dset)
        print(gen_motion().shape)
        pred_pose_eval[k:k+1,:lenn] = gen_motion()[None]

    t_latents_pred , m_latents_pred = get_latents((pred_pose_eval , inputs["motion"][1] ), conditions)
    score_pred = get_score_matrix(t_latents_pred, m_latents_pred).cpu()

    motion_list.append(m_latents)
    motion_pred_list.append(m_latents_pred)

100%|███████████████████████████████████████████████████████████████████| 72/72 [00:01<00:00, 36.78it/s]

torch.Size([240, 137])


RuntimeError: The expanded size of the tensor (253) must match the existing size (240) at non-singleton dimension 1.  Target sizes: [1, 253, 137].  Tensor sizes: [240, 137]

In [122]:
gen_motion().shape

torch.Size([240, 137])

In [123]:
pred_pose_eval[0,:240] = gen_motion()

In [ ]:
temp_R, temp_match = calculate_R_precision(
    t_latents.cpu().numpy(), m_latents.cpu().numpy(), top_k=3, sum_all=True
)
R_precision_real += temp_R
matching_score_real += temp_match
temp_R, temp_match = calculate_R_precision(
    t_latents_pred.cpu().numpy(), m_latents_pred.cpu().numpy(), top_k=3, sum_all=True
)
R_precision += temp_R
matching_score_pred += temp_match

In [223]:
def GI(split1 , split2):
    total1 = sum(split1)
    ip1 =  1 - sum([(i/total1)**2 for i in split1])
    total2 = sum(split2)
    ip2 =  1 - sum([(i/total2)**2 for i in split2])
    # n1 = len(split1)
    # n2 = len(split2)
    nt = total1 + total2
    print(ip1 , ip2)

    return (total1/nt) * ip1 + (total2/nt) * ip2

def CE(split1 , split2):
    total1 = sum(split1)
    ip1 =  1 - max([(i/total1) for i in split1])
    total2 = sum(split2)
    ip2 =  1 - max([(i/total2) for i in split2])
    # n1 = len(split1)
    # n2 = len(split2)
    nt = total1 + total2
    print(ip1 , ip2)

    return (total1/nt) * ip1 + (total2/nt) * ip2
    

In [178]:
# ## Work
# GI([15 , 20 , 10] , [15 , 10 , 20] )
# GI([20 , 10 , 20] , [10 , 20 , 10] )

0.6325

In [135]:
split = [20 , 10 , 15]
total = sum(split)
1 - sum([(i/total)**2 for i in split])

0.6419753086419753

In [134]:
split = [10 , 20 , 15]
total = sum(split)
1 - sum([(i/total)**2 for i in split])

0.6419753086419753

In [224]:
CE([30 , 30 , 30] , [30 , 30 , 30]  )

0.6666666666666667 0.6666666666666667


0.6666666666666667

In [225]:
CE([30 , 15 , 15] , [0 , 15 , 15] )

0.5 0.5


0.5

In [226]:
CE([20 , 20 , 5] , [10 , 10 , 25] )

0.5555555555555556 0.4444444444444444


0.5

In [227]:
GI([30 , 30 , 30] , [30 , 30 , 30]  )

0.6666666666666667 0.6666666666666667


0.6666666666666667

In [228]:
GI([30 , 15 , 15] , [0 , 15 , 15] )

0.625 0.5


0.5833333333333333

In [229]:
GI([20 , 20 , 5] , [10 , 10 , 25] )

0.5925925925925926 0.5925925925925926


0.5925925925925926

In [235]:
0.667 - 0.5925925925925926

0.07440740740740748

In [234]:
16/27

0.5925925925925926

In [239]:
!git clone https://github.com/EricGuo5513/momask-codes.git

Cloning into 'momask-codes'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 240 (delta 56), reused 43 (delta 43), pack-reused 162
Receiving objects: 100% (240/240), 1.14 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (111/111), done.
Updating files: 100% (77/77), done.
